# Import Libraries

In [1]:
import os
import pandas as pd
import string
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
from textblob import TextBlob
import csv
import numpy as np
import re

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Om
[nltk_data]     Ankesh1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Om
[nltk_data]     Ankesh1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
sheet_id = '1D7QkDHxUSKnQhR--q0BAwKMxQlUyoJTQ'
df = pd.read_excel(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")
df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [4]:
stop_words =[]

In [5]:
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_Auditor.txt","r") as f:
    words = f.read().split("\n") 
    stop_words.extend(words)
    
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_Currencies.txt","r") as f:
    currency = f.read().split("\n")
    words = []
    for item in currency:
        words.append(item.split('|')[0])
    stop_words.extend(words)

with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_DatesandNumbers.txt","r") as f:
    words = f.read().split("\n") 
    stop_words.extend(words)
    
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_Generic.txt","r") as f:
    words = f.read().split("\n") 
    stop_words.extend(words)

with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_GenericLong.txt","r") as f:
    words = f.read().split("\n") 
    stop_words.extend(words)
    
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_Geographic.txt","r") as f:
    words = f.read().split("\n") 
    stop_words.extend(words)
    
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/StopWords/StopWords_Names.txt","r") as f:
    words = f.read().split("\n") 
    stop_words.extend(words)

In [6]:
print(len(stop_words))

14111


In [7]:
#### Check for positive words

In [8]:
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/MasterDictionary/positive-words.txt","r") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]

In [9]:
### Check for negative words

In [10]:
with open("C:/Users/Om Ankesh1/Blackoffer Assignment/MasterDictionary/negative-words.txt","r",encoding = "ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    negwords = negwords[36:]

In [11]:
cols = ["URL_ID","URL","Title","Article","Total_words","Total_sentence","Cleaned_artical","Total_words_after_cleaning","POSITIVE SCORE","NEGATIVE SCORE"]
data=pd.DataFrame(columns=cols)

In [12]:
data

,URL_ID,URL,Title,Article,Total_words,Total_sentence,Cleaned_artical,Total_words_after_cleaning,POSITIVE SCORE,NEGATIVE SCORE


# Calculate Positive Score and Negative Score

In [13]:
for ind,row in df.iterrows():
    filename = str(row['URL_ID'])+".txt"
    url_id =row['URL_ID']
    url = row['URL']
    
    
    if os.path.isfile(filename):
        
        #for Extracting title and article
        f=open(filename, mode='r',encoding='utf-8')
        
        title = f.readline().strip('\n')
        article = f.readline()
        f.close()        
        
        
        f=open(filename, mode='r',encoding='utf-8')
        
        #Punctuation
        text = f.read()
        words = text.split()
        
        no_of_words = len(words)
        number_of_sentences = len(sent_tokenize(text))
        
        content =" ".join([w.translate(str.maketrans('', '', string.punctuation))for w in words])
        
        #Tokenization
        text_tokens = word_tokenize(content)
        #print(text_tokens[0:50])
        
        # lenghts of tokens before removing stopwords
        #print(len(text_tokens))
        
        # Remove stopwords from the tokens
        my_stop_words = stopwords.words('english')
        my_stop_words.append('the')
        my_stop_words.extend(stop_words)
        no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]
        
             
        # Check for positive words
        pos_count = " ".join ([w for w in no_stop_tokens if w in poswords])
        pos_count=pos_count.split(" ")
        #Positive Score
        Positive_score=len(pos_count)
        #print(Positive_score)
        
        # Check for negative words
        neg_count = " ".join ([w for w in no_stop_tokens if w in negwords])
        neg_count=neg_count.split(" ")
        # negative score
        Negative_score=len(neg_count)
        #print(Negative_score)
        
        filter_content = ' '.join(no_stop_tokens)
        total_word_after_cleaning = len(filter_content.split())
                  
        
        new_row = [url_id,url,title,article,no_of_words,number_of_sentences,filter_content,total_word_after_cleaning,Positive_score,Negative_score]
        
        data = pd.DataFrame(np.insert(data.values,ind,new_row,axis=0), columns =cols)

# Removing Dead Link row from dataset

In [14]:
data[data.Total_words == 0]

,URL_ID,URL,Title,Article,Total_words,Total_sentence,Cleaned_artical,Total_words_after_cleaning,POSITIVE SCORE,NEGATIVE SCORE
7,44,https://insights.blackcoffer.com/how-neural-ne...,,,0,0,,0,1,1
20,57,https://insights.blackcoffer.com/covid-19-envi...,,,0,0,,0,1,1
107,144,https://insights.blackcoffer.com/ensuring-grow...,,,0,0,,0,1,1


In [15]:
data = data.drop(data[data.Total_words == 0].index)

In [16]:
data.head()

,URL_ID,URL,Title,Article,Total_words,Total_sentence,Cleaned_artical,Total_words_after_cleaning,POSITIVE SCORE,NEGATIVE SCORE
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction “If anything kills over 10 milli...,1797,77,healthcare Improve Patient Outcomes Introducti...,1068,63,32
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying...",1420,81,What Creation Taking Over Creator Human minds ...,669,56,37
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction AI is rapidly evolving in the em...,1710,86,What Jobs Will Robots Take From Humans The Fut...,943,66,33
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-tha...,1660,96,Will Machine Replace The Human Future Work “ A...,775,57,25
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention t...,1747,80,Will Replace Us Work With Us “ Machine intelli...,932,51,25


In [17]:
data['POLARITY SCORE'] = (data['POSITIVE SCORE']-data['NEGATIVE SCORE'])/ ((data['POSITIVE SCORE'] +data['NEGATIVE SCORE']) + 0.000001)

In [18]:
data['SUBJECTIVITY SCORE'] = (data['POSITIVE SCORE'] + data['NEGATIVE SCORE'])/( data['Total_words_after_cleaning'] + 0.000001)

# Average sentence lengeth

In [19]:
data['AVG SENTENCE LENGTH'] = data['Total_words']/data['Total_sentence']

# Sylable count

In [20]:
complex_words = []
sylabble_counts = []

In [21]:
for article in data.Article:
    sylabble_count=0
    d=article.split()
    ans=0
    for word in d:
        word = word.lower()
        count=0
        for i in range(len(word)):
          if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
               count+=1
        #            print(words[i])
        if word.endswith("es") or word.endswith("ed"):
            count-=1
        sylabble_count+=count    
        if(count>2):
            ans+=1
    sylabble_counts.append(sylabble_count)
    complex_words.append(ans)

In [22]:
sylabble_counts

[3788,
 2567,
 3469,
 3049,
 3272,
 2427,
 1364,
 1294,
 4127,
 3839,
 2473,
 2333,
 3129,
 3189,
 1053,
 3268,
 1874,
 1450,
 614,
 2628,
 1332,
 3554,
 1275,
 714,
 1963,
 1292,
 762,
 331,
 1448,
 324,
 2504,
 994,
 1909,
 3479,
 3126,
 2626,
 1499,
 2423,
 871,
 3147,
 1838,
 1873,
 2084,
 2787,
 1943,
 2732,
 1448,
 2328,
 3252,
 5835,
 3298,
 3122,
 303,
 1761,
 3314,
 3308,
 1971,
 3383,
 1804,
 2235,
 2044,
 2441,
 323,
 1969,
 1141,
 2165,
 1901,
 769,
 1049,
 2027,
 1143,
 1962,
 2020,
 1991,
 2327,
 2811,
 3329,
 1373,
 3649,
 2200,
 644,
 3962,
 2688,
 3155,
 3363,
 3689,
 3496,
 1559,
 3089,
 1835,
 1297,
 3259,
 3372,
 2611,
 2162,
 796,
 540,
 674,
 406,
 490,
 1250,
 1879,
 1731,
 3474,
 1138,
 1746,
 1754,
 2951,
 2244,
 1511,
 1993]

# Calculate Percentage of complex words

In [23]:
data['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/data['Total_words']

# Fog Index

In [24]:
data['FOG INDEX'] = 0.4 * (data['AVG SENTENCE LENGTH'] + data['PERCENTAGE OF COMPLEX WORDS'])

# Average Number of Words Per Sentence

In [25]:
data['AVG NUMBER OF WORDS PER SENTENCES'] = data['AVG SENTENCE LENGTH']

# Complex Word Count

In [26]:
data['COMPLEX WORD COUNT'] = complex_words

# Word Count

In [27]:
data['WORD COUNT'] = data['Total_words_after_cleaning']

# Syllable Count Per Word

In [28]:
data['SYLLABLE PER WORD'] = np.array(sylabble_counts)/data['Total_words']

# Personal Pronouns

In [29]:
total_characters = []
for article in data.Article:
  characters = 0
  for word in article.split():
    characters+=len(word)
  total_characters.append(characters) 

In [30]:
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And'] 
for article in data.Article:
  ans=0
  for word in article:
    if word in personal_noun:
      ans+=1
  personal_nouns.append(ans)

In [31]:
data['PERSONAL PRONOUN'] = personal_nouns

# Average Word Length

In [32]:
data['AVG WORD LENGTH'] = np.array(total_characters)/data['Total_words']

In [33]:
data.head()

,URL_ID,URL,Title,Article,Total_words,Total_sentence,Cleaned_artical,Total_words_after_cleaning,POSITIVE SCORE,NEGATIVE SCORE,...,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,Introduction “If anything kills over 10 milli...,1797,77,healthcare Improve Patient Outcomes Introducti...,1068,63,32,...,0.088951,23.337662,0.331107,9.467508,23.337662,595,1068,2.107958,39,5.777407
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"Human minds, a fascination in itself carrying...",1420,81,What Creation Taking Over Creator Human minds ...,669,56,37,...,0.139013,17.530864,0.214789,7.098261,17.530864,305,669,1.807746,37,4.975352
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction AI is rapidly evolving in the em...,1710,86,What Jobs Will Robots Take From Humans The Fut...,943,66,33,...,0.104984,19.883721,0.307018,8.076295,19.883721,525,943,2.028655,31,5.507018
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,“Anything that could give rise to smarter-tha...,1660,96,Will Machine Replace The Human Future Work “ A...,775,57,25,...,0.105806,17.291667,0.218072,7.003896,17.291667,362,775,1.836747,74,4.887349
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us?,“Machine intelligence is the last invention t...,1747,80,Will Replace Us Work With Us “ Machine intelli...,932,51,25,...,0.081545,21.8375,0.236978,8.829791,21.8375,414,932,1.872925,36,5.225529


In [34]:
data.columns

Index(['URL_ID', 'URL', 'Title', 'Article', 'Total_words', 'Total_sentence',
       'Cleaned_artical', 'Total_words_after_cleaning', 'POSITIVE SCORE',
       'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
       'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCES', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUN', 'AVG WORD LENGTH'],
      dtype='object')

In [35]:
Final_data = data[['URL_ID', 'URL', 'POSITIVE SCORE',
       'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
       'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCES', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUN', 'AVG WORD LENGTH']]

In [36]:
Final_data.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,63,32,0.326316,0.088951,23.337662,0.331107,9.467508,23.337662,595,1068,2.107958,39,5.777407
1,38,https://insights.blackcoffer.com/what-if-the-c...,56,37,0.204301,0.139013,17.530864,0.214789,7.098261,17.530864,305,669,1.807746,37,4.975352
2,39,https://insights.blackcoffer.com/what-jobs-wil...,66,33,0.333333,0.104984,19.883721,0.307018,8.076295,19.883721,525,943,2.028655,31,5.507018
3,40,https://insights.blackcoffer.com/will-machine-...,57,25,0.390244,0.105806,17.291667,0.218072,7.003896,17.291667,362,775,1.836747,74,4.887349
4,41,https://insights.blackcoffer.com/will-ai-repla...,51,25,0.342105,0.081545,21.8375,0.236978,8.829791,21.8375,414,932,1.872925,36,5.225529


# Exporting data in csv file 

In [38]:
Final_data.to_csv('Output Data Structure.csv', index=False)